# Charts

In [1]:
import numpy as np
import pandas as pd
from siuba import *
from calitp import *
import intake
import data_prep
import _utils
import shared_utils
import altair as alt
import altair_saver
from shared_utils import altair_utils 
pd.options.display.max_rows = 100
pd.options.display.float_format = "{:.0f}".format
pd.options.display.float_format = '{:,}'.format

/opt/conda/lib/python3.9/site-packages/geopandas/_compat.py:111: UserWarning: The Shapely GEOS version (3.9.1-CAPI-1.14.2) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.


In [2]:
BC_GTFS_NTD2 = pd.read_parquet('BC_GTFS_NTD.parquet')

In [3]:
BC_test = data_prep.merged_dataframe()

/home/jovyan/data-analyses/5311/data_prep.py:68: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/jovyan/data-analyses/5311/data_prep.py:70: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/jovyan/data-analyses/5311/data_prep.py:122: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-ve

In [4]:
#Grabbing only one row for each agency.
Aggregate1 = BC_GTFS_NTD2.groupby(['organization_name','reporter_type','GTFS', 'fleet_size', 'Is_Agency_In_BC_Only_1_means_Yes']).agg({'total_vehicles':'max',
       'average_age_of_fleet__in_years_':'max','doors_sum':'max','_0_9':'max', '_10_12':'max', '_13_15':'max', '_16_20':'max', '_21_25':'max',
       '_26_30':'max', '_31_60':'max', '_60plus':'max',})

In [5]:
#Reset index
Aggregate1 = Aggregate1.reset_index()
Aggregate1


,organization_name,reporter_type,GTFS,fleet_size,Is_Agency_In_BC_Only_1_means_Yes,total_vehicles,average_age_of_fleet__in_years_,doors_sum,_0_9,_10_12,_13_15,_16_20,_21_25,_26_30,_31_60,_60plus
0,Alpine County Community Development,None,None,No Info,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Amador Regional Transit System,Rural Reporter,Ok,Medium,0,24.0,4.065359477333334,21.0,18.0,3.0,1.0,1.0,1.0,0.0,0.0,0.0
2,Butte County Association of Governments,Full Reporter,Ok,Medium,0,53.0,6.228448276,82.0,44.0,9.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Calaveras Transit Agency,Rural Reporter,None,Medium,0,12.0,5.0,11.0,12.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,"Central Contra Costa Transit Authority, dba: C...",Full Reporter,None,Large,0,206.0,3.4399548023333337,312.0,160.0,43.0,1.0,1.0,1.0,0.0,0.0,0.0
5,City of Arcata,Rural Reporter,Ok,Medium,0,10.0,8.5,12.0,5.0,4.0,1.0,0.0,0.0,0.0,0.0,0.0
6,City of Arvin,Rural Reporter,Long-term solution needed,Medium,0,10.0,6.583333333000001,15.0,9.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
7,City of Auburn,Rural Reporter,Ok,Small,0,6.0,6.25,7.0,5.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
8,City of California City,Rural Reporter,Needed,Small,0,4.0,11.0,8.0,0.0,3.0,1.0,0.0,0.0,0.0,0.0,0.0
9,"City of Chowchilla, dba: Chowchilla Area Transit",Rural Reporter,Needed,Small,0,4.0,5.75,4.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


#  GTFS Status Charts
* One for vehicles over 9
* One for vehicles over 15

In [6]:
vehicles_gtfs = _utils.aggregation_one(Aggregate1, 'GTFS')
vehicles_gtfs

,GTFS,vehicles_older_than_9,vehicles_older_than_15,vehicles_0_to_9,vehicles_percent_older_than_9,vehicles_percent_older_than_15,vehicles_percent_0_to_9
0,Long-term solution needed,7.0,1.0,24.0,0.34347399411187435,0.13351134846461948,0.4840661557079467
1,Needed,34.0,7.0,122.0,1.6683022571148183,0.9345794392523363,2.4606696248487294
2,None,51.0,5.0,191.0,2.502453385672228,0.6675567423230975,3.8523598225090763
3,Ok,"1,901.0",720.0,"4,470.0",93.27772325809617,96.12817089452604,90.15732150060508
4,Research,45.0,16.0,151.0,2.2080471050049066,2.1361815754339117,3.0455828963291647


### Looking at vehicles 9+ by GTFS
![pic](./chart_outputs/bar_GTFS_by_vehicles_older_than_9.png)

In [7]:
_utils.basic_bar_chart(vehicles_gtfs,'GTFS','vehicles_older_than_9','GTFS') 

alt.Chart(...)

In [8]:
_utils.basic_bar_chart(vehicles_gtfs,'GTFS','vehicles_percent_older_than_9','GTFS') 

alt.Chart(...)

### Looking at vehicles 15+ by GTFS
![pic](./chart_outputs/bar_GTFS_by_vehicles_older_than_15.png)

In [9]:
_utils.basic_bar_chart(vehicles_gtfs,'GTFS','vehicles_older_than_15', 'GTFS') 

alt.Chart(...)

# Fleet size 

In [10]:
fleet = _utils.aggregation_one(Aggregate1,'fleet_size') 
fleet

,fleet_size,vehicles_older_than_9,vehicles_older_than_15,vehicles_0_to_9,vehicles_percent_older_than_9,vehicles_percent_older_than_15,vehicles_percent_0_to_9
0,Large,"1,757.0",696.0,"3,975.0",86.21197252208047,92.92389853137517,80.17345703912868
1,Medium,255.0,48.0,903.0,12.512266928361138,6.408544726301736,18.212989108511497
2,No Info,0.0,0.0,0.0,0.0,0.0,0.0
3,Small,26.0,5.0,80.0,1.2757605495583906,0.6675567423230975,1.6135538523598225


In [11]:
# don't need no info
fleet =  fleet.loc[fleet['fleet_size'] != 'No Info'] 

### Vehicles older than 9
![pic](./chart_outputs/bar_fleet_size_by_vehicles_older_than_9.png)

In [12]:
_utils.basic_bar_chart(fleet,'fleet_size','vehicles_older_than_9', 'fleet_size') 

alt.Chart(...)

### Vehicles older than 15
![pic](./chart_outputs/bar_fleet_size_by_vehicles_older_than_15.png)

In [13]:
_utils.basic_bar_chart(fleet,'fleet_size','vehicles_older_than_15', 'fleet_size') 

alt.Chart(...)

# Other
## Looking at organizations & GTFS
![pic](./chart_outputs/bar_GTFS_by_count_of_agencies.png)

In [14]:
Orgs = Aggregate1.groupby(['GTFS']).agg({'organization_name':'nunique'})
Orgs = Orgs.reset_index()
Orgs = Orgs.rename(columns = {'organization_name': 'Organizations'}) 
_utils.basic_bar_chart(Orgs,'GTFS','Organizations', 'GTFS') 
Orgs.sort_values(by=['Organizations'])

,GTFS,Organizations
4,Research,1
0,Long-term solution needed,3
2,None,14
1,Needed,15
3,Ok,54


### Fleet Size by agencies

In [15]:
Fleet = Aggregate1.groupby(['fleet_size']).agg({'organization_name':'nunique'}).reset_index()
Fleet

,fleet_size,organization_name
0,Large,20
1,Medium,38
2,No Info,9
3,Small,20


## Looking at reporter type & GTFS
![testing](./chart_outputs/bar_GTFS_by_count_of_agencies.png)

![testing](./chart_outputs/bar_reporter_type_by_Count of Agencies.png)

In [16]:
Reporter_type_agg = Aggregate1.groupby(['reporter_type']).agg({'organization_name':'nunique'})
Reporter_type_agg = Reporter_type_agg.reset_index()
Reporter_type_agg = Reporter_type_agg.rename(columns = {'organization_name':'Count_of_Agencies'})
_utils.basic_bar_chart(Reporter_type_agg,'reporter_type','Count_of_Agencies', 'reporter_type') 

alt.Chart(...)

In [17]:
fleet_size_avg_age = Aggregate1.groupby(['fleet_size']).agg({'average_age_of_fleet__in_years_':'mean'})
fleet_size_avg_age

,average_age_of_fleet__in_years_
fleet_size,
Large,7.971572030532262
Medium,6.5403448314342105
No Info,NaN
Small,6.9774999999125


In [18]:
Aggregate1.columns

Index(['organization_name', 'reporter_type', 'GTFS', 'fleet_size',
       'Is_Agency_In_BC_Only_1_means_Yes', 'total_vehicles',
       'average_age_of_fleet__in_years_', 'doors_sum', '_0_9', '_10_12',
       '_13_15', '_16_20', '_21_25', '_26_30', '_31_60', '_60plus',
       'vehicles_older_than_9', 'vehicles_older_than_15'],
      dtype='object')

In [19]:
age = Aggregate1
age  =  age.loc[age['fleet_size'] != 'No Info'] 
age = age.groupby(['GTFS','fleet_size']).agg({'_0_9':'sum', 'vehicles_older_than_9':'sum','average_age_of_fleet__in_years_':'mean'})
age

_0_9  vehicles_older_than_9  \
GTFS                      fleet_size                                  
Long-term solution needed Medium        24.0                    7.0   
Needed                    Medium        87.0                   12.0   
                          Small         35.0                   22.0   
None                      Large        160.0                   46.0   
                          Medium        19.0                    5.0   
                          Small         12.0                    0.0   
Ok                        Large      3,664.0                1,666.0   
                          Medium       773.0                  231.0   
                          Small         33.0                    4.0   
Research                  Large        151.0                   45.0   

                                      average_age_of_fleet__in_years_  
GTFS                      fleet_size                                   
Long-term solution needed Medium                    8.937499999833333  
Needed                    Medium                         8.2915140415  
                          Small                     8.498333333175001  
None                      Large                    3.4399548023333337  
                          Medium                    6.199999999999999  
                          Small                     5.277777777666667  
Ok                        Large                      8.27016978176733  
                          Medium                    6.074294738931034  
                          Small                     5.533333333357143  
Research                  Large                     7.128429736499999

## Looking at agencies that spent the most..
![pic](./chart_outputs/bar_organization_name_by_allocationamount.png)

In [20]:
Most_Money = BC_GTFS_NTD2.groupby(['organization_name','GTFS']).agg({'allocationamount':'sum'})

In [21]:
Top_ten = Most_Money.sort_values('allocationamount').tail(10)
Top_ten =Top_ten.rename(columns = {'allocationamount': '5311_funds_received'})
Top_ten

,,5311_funds_received
organization_name,GTFS,
Transit Joint Powers Authority for Merced County,Ok,"3,216,038.0"
Mountain Area Regional Transit Authority,Ok,"3,833,588.0"
Victor Valley Transit Authority,Ok,"3,990,601.0"
Mendocino Transit Authority,Ok,"4,203,816.0"
Lake Transit Authority,Ok,"5,055,140.0"
Humboldt Transit Authority,Ok,"5,717,828.0"
Butte County Association of Governments,Ok,"5,886,587.0"
Kern Regional Transit,Ok,"7,496,759.0"
Monterey-Salinas Transit,Research,"8,385,671.0"


In [22]:
Most_Money.sort_values('allocationamount')

,,allocationamount
organization_name,GTFS,
"Central Contra Costa Transit Authority, dba: COUNTY CONNECTION",None,"50,412.0"
Livermore / Amador Valley Transit Authority,Ok,"87,743.0"
Klamath Trinity Non-Emergency Transportation​,Needed,"92,345.0"
City of Taft,Long-term solution needed,"186,591.0"
City of Fairfield,Ok,"200,000.0"
"City of Escalon, dba: eTrans",None,"214,476.0"
City of Needles,Ok,"240,965.0"
City of Tehachapi,None,"244,117.0"
City of California City,Needed,"276,006.0"


In [23]:
BC_GTFS_NTD2.columns

Index(['grant_fiscal_year', 'funding_program', 'grant_number', 'project_year',
       'organization_name', 'upin', 'description', 'ali', 'contract_number',
       'allocationamount', 'encumbered_amount', 'expendedamount',
       'activebalance', 'closedoutbalance', 'project_status', 'ntd_id',
       'reporter_type', '_0_9', '_10_12', '_13_15', '_16_20', '_21_25',
       '_26_30', '_31_60', '_60plus', 'total_vehicles',
       'average_age_of_fleet__in_years_', 'average_lifetime_miles_per_vehicle',
       'sum_15plus', 'Automobiles', 'Bus', 'Other', 'Service', 'Train', 'Van',
       'automobiles_door', 'bus_doors', 'train_doors', 'van_doors',
       'doors_sum', 'itp_id', 'Is_Agency_In_BC_Only_1_means_Yes', 'GTFS',
       'fleet_size'],
      dtype='object')

In [24]:
bc_funds = BC_GTFS_NTD2.groupby(['funding_program']).agg({'organization_name':'nunique','allocationamount':'sum'})
bc_funds = bc_funds.rename(columns = {'organization_name':'Count_of_Organizations','allocationamount':'total_sum'})
bc_funds.sort_values(by =['total_sum'])

,Count_of_Organizations,total_sum
funding_program,,
CMAQ (FTA 5311),6,"2,532,205.0"
5311(f) Round 2,20,"6,203,361.0"
5311(f) Cont,21,"7,627,280.0"
Section 5311(f),22,"18,753,232.0"
Section 5311,81,"129,415,424.28"
